In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Simulation settings
dtime = 0.01 # time step
iterative = True
ideal_diodes = False

# Other
diode_volt_drop = 0 if ideal_diodes else 0.7

# Capacitor
capacitance = 1e-4 # farads
#phase = np.pi / 2 # 90 degrees, TODO: change to calculate based on resistance

# Zener diode
has_zener = True
zener_voltage = 9

# Resistor
resistance = 800 # ohm

# Source
frequency = 60 # hertz
voltage = 13 # volts
current_volt = lambda t: np.sin(2*np.pi*t*frequency) * voltage# * voltage # function given time in seconds

# Rectifiers

# False is single diode, True is diode bridge
diode_bridge = True

if diode_bridge:
    rect_volt = lambda volt: np.abs(volt) if ideal_diodes else np.max(np.abs(volt) - diode_volt_drop * 2, 0)
else:
    rect_volt = lambda volt: volt if volt - (diode_volt_drop * ideal_diodes)>= 1e-14 else 0

if has_zener:
    def limited_volt(volt): # disconsiders negative voltage
        if volt > zener_voltage:
            return zener_voltage
        else:
            return volt
else:
    limited_volt = lambda volt: volt

In [ ]:
# Calculating functions
def charge_capacitor(time_step, current_charge, applied_volt):
    time_const = capacitance * resistance
    
    next_charge = applied_volt + (current_charge - applied_volt) * np.exp(-time_step/time_const)
    
    return next_charge

In [ ]:
times = []
values = []

if iterative:
    cap_charge = 0
    t = 0
    
    while t < 10:
        volts = limited_volt(rect_volt(current_volt(t)))
        
        cap_charge = charge_capacitor(dtime, cap_charge, volts)
        values.append(cap_charge)
        times.append(t)
        
        t += dtime
        
plt.plot(times, values) #marker="o"

# Adding labels and legend
plt.xlabel('Time')
plt.ylabel('Voltage across capacitor')
plt.title('')

# Display the plot
plt.show()